In [ ]:
import pandas as pd

In [21]:
campaigns = pd.read_csv('../../data/processed/campaigns.csv')
customers = pd.read_csv('../../data/processed/customer.csv')
engagements = pd.read_csv('../../data/processed/engagement_details.csv')

## Campaign personalisation scores

The personalisation scores for each campaign reflect how tailored and relevant a campaign is for a given target audience. A higher personalisation score means that the campaign is better suited for specific audience segments, offering mroe customised content and messaging.

In [ ]:
def get_personalisation_score(row):
    campaign_type_weights = {
        'Search Engine Optimisation': 0.2,
        'Email Marketing': 0.8,
        'Affiliate Marketing': 0.5,
        'Display Advertising': 0.3
    }

    language_score = 0.5 if row['campaign_language'] in ['Mandarin', 'French', 'Spanish', 'German'] else 0.3

    if row['target_audience'] == '35-44':
        target_audience_score = 0.7
    elif row['target_audience'] == '55+':
        target_audience_score = 0.6
    else:
        target_audience_score = 0.5

    if row['campaign_duration'] > 60:
        duration_score = 0.8
    elif row['campaign_duration'] > 30:
        duration_score = 0.6
    else:
        duration_score = 0.4

    campaign_type_score = campaign_type_weights.get(row['campaign_type'], 0.4)

    personalisation_score = (
        0.25 * campaign_type_score +
        0.2 * language_score + 
        0.25 * target_audience_score +
        0.3 * duration_score
    )

    return personalisation_score


campaigns['personalisation_score'] = campaigns.apply(get_personalisation_score, axis=1)

## Campaign engagement scores

The engagement scores for each campaign indicate their effectiveness – how well the campaign has succeeded in attracting interactions from its audience. Higher engagement scores suggest campaigns have resonated well with customers and elicited more significant actions, such as clicks and views.

In [ ]:
merged = pd.merge(engagements, campaigns, on='campaign_id', how='left')

campaign_engagements = merged.groupby('campaign_id').agg(
    total_engagements=('has_engaged', 'sum'),
    total_duration=('duration', 'sum'),
    impressions=('impressions', 'first')
).reset_index()

campaign_engagements['engagement_rate'] = campaign_engagements['total_engagements'] / campaign_engagements['impressions']
campaign_engagements['effective_engagement_rate'] = campaign_engagements['total_duration'] / campaign_engagements['impressions']

campaign_engagements['campaign_effectiveness_score'] = (
    0.5 * campaign_engagements['engagement_rate'] +
    0.5 * campaign_engagements['effective_engagement_rate']
)

campaigns_final = pd.merge(campaigns, campaign_engagements[['campaign_id', 'campaign_effectiveness_score']], on='campaign_id', how='left')